In [1]:
import spectfbcalc_lib as sfc
from climtools import climtools_lib as ctl

No DISPLAY variable set. Switching to agg backend


In [82]:
from importlib import reload
reload(sfc)

<module 'spectfbcalc_lib' from '/home/rotoli/codici/SpectFbCalc/spectfbcalc/spectfbcalc_lib.py'>

In [12]:
sfc.mytestfunction()

test!


In [3]:
ctl.datestamp()

'2024-11-12T13:22:36'

In [2]:
import sys
import os
import glob

import numpy as np
import xarray as xr

from climtools import climtools_lib as ctl
from matplotlib import pyplot as plt
import matplotlib.cbook as cbook
from scipy import stats
import pickle
import dask.array as da


In [3]:
cart_out = '/home/rotoli/codici/test3/'
ctl.mkdir(cart_out)

cart_in = '/nas/archive_CMIP6/CMIP6/model-output/EC-Earth-Consortium/EC-Earth3/'
filin_pi = cart_in + 'piControl/atmos/Amon/r1i1p1f1/{}/*nc'
filin_4c = cart_in + 'abrupt-4xCO2/atmos/Amon/r8i1p1f1_r25/{}/*nc'
filin_4c1 = cart_in + 'abrupt-4xCO2/atmos/Amon/r8i1p1f1/{}/*nc'
cart_k = '/data-hobbes/fabiano/radiative_kernels/Huang/toa/'
use_climatology=True
fb_coef=dict()

dir='/data-hobbes/fabiano/radiative_kernels/ps_ece3/ps_Amon_EC-Earth3_stabilization-hist-1990_r1i1p1f1_gr_199?01-199?12.nc'


In [4]:
allkers=dict()
ker= 'HUANG'
#cart_k="/data-hobbes/fabiano/radiative_kernels/Huang_ERA5/"
allkers=sfc.load_kernel(ker, cart_k, cart_out)

In [8]:
ds=dict()
k=allkers[('cld', 't')]
nomi='ts rsut rlut rsus rsds ta tas hus'.split()
for nom in nomi:
 filist = glob.glob(filin_4c.format(nom))
 filist.sort()
 ds[nom]= xr.open_mfdataset(filist, chunks = {'time': 12}, use_cftime=True)

nomi='rlutcs rsutcs'.split()
for nom in nomi:
 filist = glob.glob(filin_4c1.format(nom))
 filist.sort()
 ds[nom]= xr.open_mfdataset(filist, chunks = {'time': 12}, use_cftime=True)
 ds[nom] = ctl.regrid_dataset(ds[nom], k.lat, k.lon)

pressure= xr.open_mfdataset(dir)

Regridding...
Regridding completed in 0:00:01.151950
Regridding...
Regridding completed in 0:00:00.994187


In [58]:
time_range=['2540-01-01', '2689-12-31']
piok=dict()
allvars= 'ts tas hus alb ta'.split()
for vnams in allvars:  
        piok[vnams]=sfc.climatology(filin_pi, allkers, vnams, time_range, False)
allvars1= 'rlutcs rsutcs rlut rsut'.split()
for vnams in allvars1:  
        piok[vnams]=sfc.climatology(filin_pi, allkers, vnams)

Regridding...
Regridding completed in 0:00:00.966227
Regridding...
Regridding completed in 0:00:00.896325
Regridding...
Regridding completed in 0:00:00.930090
Regridding...
Regridding completed in 0:00:00.929036
Regridding...
Regridding completed in 0:00:00.944196
Regridding...
Regridding completed in 0:00:00.936139
Regridding...
Regridding completed in 0:00:00.956514
Regridding...
Regridding completed in 0:00:00.940886
Regridding...
Regridding completed in 0:00:00.949300
Regridding...
Regridding completed in 0:00:00.941828
Regridding...
Regridding completed in 0:00:00.942357


In [ ]:
pickle.dump(piok, open(cart_out + 'piok_HUANG.p', 'wb'))

: 

In [6]:
piok=pickle.load(open(cart_out + 'piok_HUANG.p', 'rb'))

In [7]:
k=allkers[('cld', 't')]
cose=pickle.load(open(cart_out + 'cose_'+ker+'.p', 'rb'))
radiation=dict()
cos="_21yearmean"
time_range=['1850-01-01', '1999-12-31']
use_climatology=False
ker='HUANG'
    
if time_range is not None:
        ds['hus'] = ds['hus'].sel(time = slice(time_range[0], time_range[1])) 
        ds['ta'] = ds['ta'].sel(time = slice(time_range[0], time_range[1]))
var = ctl.regrid_dataset(ds['hus'], k.lat, k.lon)
var_ta=ds['ta']
mask=sfc.mask_atm(var_ta['ta'])
print('mask atmo fatta')

Rv = 487.5 # gas constant of water vapor
Lv = 2.5e+06 # latent heat of water vapor

if use_climatology==False:
        sfc.check_data(ds['ta'], piok['ta'])
        piok_hus=piok['hus'].drop('time')
        piok_hus['time'] = var['time']
        piok_ta=piok['ta'].drop('time')
        piok_ta['time'] = var['time']
print('piok aggiustato')
 
ta_abs_pi = piok_ta.interp(plev = cose)
var_int = (var['hus']*mask).interp(plev = cose)
piok_int = piok_hus.interp(plev = cose)
  
if ker=='HUANG':
        wid_mask=sfc.mask_pres(pressure, cart_out, allkers)
        anoms_ok3 = xr.apply_ufunc(lambda x, mean: np.log(x) - np.log(mean), var_int, piok_int , dask = 'allowed')
        coso3= anoms_ok3*sfc.dlnws(ta_abs_pi)
print('coso calcolato')

for tip in ['clr','cld']:
        print(tip)
        kernel_lw = allkers[(tip, 'wv_lw')]
        kernel_sw = allkers[(tip, 'wv_sw')]
        kernel = kernel_lw + kernel_sw
        
        dRt = (coso3.groupby('time.month')* kernel* wid_mask/100).sum('player').groupby('time.year').mean('time')


Regridding...
Regridding completed in 0:00:00.329107
mask atmo fatta
piok aggiustato
Regridding...
Regridding completed in 0:00:01.017555
coso calcolato
clr


/home/rotoli/miniconda3/envs/ctl4/lib/python3.11/site-packages/xarray/core/indexing.py:1446: PerformanceWarning: Slicing with an out-of-order index is generating 150 times more chunks
  return self.array[key]


cld


/home/rotoli/miniconda3/envs/ctl4/lib/python3.11/site-packages/xarray/core/indexing.py:1446: PerformanceWarning: Slicing with an out-of-order index is generating 150 times more chunks
  return self.array[key]


In [9]:
radiation=dict()
dRt_glob = ctl.global_mean(dRt)
wv= dRt_glob.compute()
radiation[(tip, 'water-vapor')]=wv
wv.to_netcdf(cart_out+ "dRt_water-vapor_global_" +tip+cos +"-"+ker+"kernels.nc", format="NETCDF4")
   

: 

In [ ]:
time_range=['1850-01-01', '1999-12-31']
sfc.Rad_anomaly_wv(ds, piok, cart_out, 'HUANG', allkers, pressure, time_range, False)

Regridding...
Regridding completed in 0:00:00.177650
Regridding...
Regridding completed in 0:00:01.022165


/home/rotoli/miniconda3/envs/ctl4/lib/python3.11/site-packages/xarray/core/indexing.py:1446: PerformanceWarning: Slicing with an out-of-order index is generating 150 times more chunks
  return self.array[key]


In [9]:
#time_range=['1850-01-01', '1999-12-31']
fb_coef, fb_cloud, fb_cloud_err =sfc.calc_fb(ds, piok, 'HUANG', allkers, cart_out, pressure)

_climatology
planck surf
planck atm
albedo
w-v
Regridding...
Regridding completed in 0:00:00.162747


/home/rotoli/miniconda3/envs/ctl4/lib/python3.11/site-packages/xarray/core/indexing.py:1446: PerformanceWarning: Slicing with an out-of-order index is generating 165 times more chunks
  return self.array[key]


calcolo feedback


In [ ]:
ker= 'ERA5'
cart_k="/data-hobbes/fabiano/radiative_kernels/Huang_ERA5/"

fb_coef=sfc.calc_fb(filin_4c, filin_pi, cart_out, ker, cart_k, dir, True, 12)

In [11]:
fb_cloud

nan

In [5]:
fb_coef2=dict()
fb_coef2=sfc.calc_fb(filin_4c, filin_pi, cart_out, 'HUANG', cart_k, dir, True, 12)

_climatology
planck surf
planck atm
albedo
w-v
calcolo feedback


In [22]:
cos="_climatology"
Rv = 487.5 # gas constant of water vapor
Lv = 2.5e+06 # latent heat of water vapor
ta_abs_pi=xr.open_dataarray(cart_out+"ta_abs_pi"+cos+"-"+ker+"kernels.nc",  chunks = {'time': 12},  use_cftime=True)
gioco=( sfc.dlnws(ta_abs_pi) -((ta_abs_pi**2) * Rv/Lv))
sec=(ctl.global_mean(sfc.dlnws(ta_abs_pi)[3]))
pr=(ctl.global_mean((((ta_abs_pi**2) * Rv/Lv))[3]))
gra=ctl.global_mean(gioco[3])

In [36]:
valori = np.arange(210, 291)
temp = xr.DataArray(valori, dims=["ta"], coords={"ta": np.arange(len(valori))})
sec=(sfc.dlnws(temp))
pr=(((temp**2) * Rv/Lv))
gra=pr-sec

In [37]:
fig = plt.figure(figsize=(7,5))
gra.plot(label='diff')
sec.plot(label='dlnws')
pr.plot(label='t^2')
plt.legend()
plt.savefig('prova dlnws (valori lineari).pdf')

In [10]:
#grafico feedback
from matplotlib import pyplot as plt

#ac2 = (fb_coef2[('cld', 'planck-surf')].slope + fb_coef2[('cld', 'planck-atmo')].slope)
#data2 = [ac2, fb_coef2[('cld','lapse-rate')].slope, fb_coef2[('cld', 'water-vapor')].slope, fb_coef2[('cld', 'albedo')].slope]
ac = (fb_coef[('cld', 'planck-surf')].slope + fb_coef[('cld', 'planck-atmo')].slope)
data = [ac, fb_coef[('cld','lapse-rate')].slope, fb_coef[('cld', 'water-vapor')].slope, fb_coef[('cld', 'albedo')].slope, fb_cloud]
#err2 = [(fb_coef2[('cld', 'planck-surf')].stderr + fb_coef2[('cld', 'planck-atmo')].stderr), fb_coef2[('cld','lapse-rate')].stderr, fb_coef2[('cld', 'water-vapor')].stderr, fb_coef2[('cld', 'albedo')].stderr]
data1 =[-3.24, -0.22,  1.72,  0.58, 0.29]
err = [(fb_coef[('cld', 'planck-surf')].stderr + fb_coef[('cld', 'planck-atmo')].stderr), fb_coef[('cld','lapse-rate')].stderr, fb_coef[('cld', 'water-vapor')].stderr, fb_coef[('cld', 'albedo')].stderr, fb_cloud_err]
err1 = [0.05, 0.18, 0.14, 0.09, 0.36]
fbnams1 = ['planck', 'lapse-rate', 'water-vapor', 'albedo', 'cloud']

fig = plt.figure(figsize=(7,5))
offset=0.05
plt.errorbar(range(len(fbnams1)), data, yerr=err, marker=".", linestyle= 'None', label='climatology', color='royalblue')
plt.errorbar([x+offset for x in range(len(fbnams1))], data1, yerr=err1, marker=".", linestyle= 'None', label='by Zelinka et al. 2020', color='navy')
#plt.errorbar([x-offset for x in range(len(fbnams1))], data2, yerr=err2, marker=".", linestyle= 'None', label='21yearmean', color='b')
plt.xticks(range(len(fbnams1)), fbnams1)
plt.legend(loc='upper left')
plt.ylabel('[W $m^{-2}$ $K^{-1}$]')

plt.savefig(cart_out +'feedback_test(climatology_HUANG).pdf')